########## choose stock list (aka market) ##################

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [2]:
## To acquire stock list in thailand ###
ex = pd.read_excel('stock_info_th.xlsx', sheet_name='listedCompanies_th_TH',skiprows=1)
df = pd.DataFrame(ex['หลักทรัพย์'].astype('str') + '.bk')
df['market'] = ex['ตลาด']
df = df.rename(columns={'หลักทรัพย์' : 'ticker'})
df = df.reset_index(drop=True)
# df = df[:50]

### Acquire informations from yfinance function >> .info

In [3]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [4]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'enterpriseValue',
    'currentPrice',
    'totalCashPerShare',
    'profitMargins',
    'trailingPE'
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT',
    'Operating Income'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_latest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


##### Drop NAN #############
df = df.dropna()
##### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.loc[df['Operating Income'] >0]
df = df.reset_index(drop=True)

df.to_csv('data_stock_th.csv',index= False)

############################################################################

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_csv('data_stock_th.csv')
print('sector', df['sector'].unique())
print('industry', df['industry'].unique())

sector ['Basic Materials' 'Industrials' 'Real Estate' 'Consumer Defensive'
 'Utilities' 'Consumer Cyclical' 'Communication Services' 'Healthcare'
 'Technology' 'Financial Services' 'Energy']
industry ['Steel' 'Electrical Equipment & Parts' 'Real Estate - Development'
 'Packaged Foods' 'Airlines' 'Utilities - Renewable'
 'Auto & Truck Dealerships' 'Advertising Agencies' 'Telecom Services'
 'Auto Parts' 'Medical Care Facilities' 'Farm Products' 'REIT - Retail'
 'REIT - Industrial' 'Information Technology Services' 'Waste Management'
 'Industrial Distribution' 'REIT - Diversified' 'Communication Equipment'
 'Aluminum' 'Marine Shipping' 'Credit Services'
 'Scientific & Technical Instruments' 'Publishing' 'Real Estate Services'
 'Integrated Freight & Logistics' 'Airports & Air Services'
 'Household & Personal Products' 'Engineering & Construction'
 'Software - Application' 'Specialty Business Services'
 'Building Products & Equipment' 'Electronic Gaming & Multimedia'
 'Aerospace & Defense' 

In [6]:

### Next I perform calculations for MF_ROC and MF_EY ###
represents_earning = 'Operating Income'

df['MF_ROC'] = df[represents_earning]/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df[represents_earning]/df['enterpriseValue']
df

,ticker,market,date_pulling,industry,sector,enterpriseValue,currentPrice,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY
0,2S.bk,SET,2024-08-20,Steel,Basic Materials,1.336299e+09,2.94,0.498,0.01822,12.782609,2.400035e+09,3.039420e+08,1.225020e+08,1.041320e+08,2024-03-31,0.049679,0.077926
1,3K-BAT.bk,SET,2024-08-20,Electrical Equipment & Parts,Industrials,3.713366e+09,53.00,10.383,0.04918,17.377050,3.874473e+09,9.397630e+08,3.250499e+08,1.733523e+08,2024-06-30,0.059070,0.046683
2,A5.bk,mai,2024-08-20,Real Estate - Development,Real Estate,4.663804e+09,2.88,0.344,0.37066,5.333334,3.775574e+09,1.024737e+09,7.916811e+08,5.779435e+08,2024-06-30,0.210097,0.123921
3,AAI.bk,SET,2024-08-20,Packaged Foods,Consumer Defensive,1.202628e+10,6.10,0.468,0.13508,16.052631,5.828192e+09,5.910240e+08,9.276925e+08,8.961244e+08,2024-06-30,0.171109,0.074514
4,AAV.bk,SET,2024-08-20,Airlines,Industrials,7.301899e+10,2.32,0.125,0.01645,33.142857,7.252696e+10,2.862134e+10,3.279661e+09,3.208543e+09,2024-06-30,0.073078,0.043941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,YGG.bk,mai,2024-08-20,Entertainment,Communication Services,6.950683e+08,0.86,0.049,0.22043,7.166667,7.375050e+08,7.321346e+07,6.699039e+07,6.147763e+07,2024-03-31,0.092546,0.088448
566,YONG.bk,mai,2024-08-20,Building Materials,Basic Materials,1.302733e+09,1.33,0.065,0.11048,7.388889,1.304679e+09,3.079860e+08,1.533742e+08,1.510244e+08,2024-06-30,0.151525,0.115929
567,YUASA.bk,mai,2024-08-20,Auto Parts,Consumer Cyclical,7.521635e+08,10.80,3.881,0.06399,7.058824,1.684253e+09,4.092886e+08,1.939366e+08,1.814172e+08,2024-06-30,0.142292,0.241194
568,ZEN.bk,SET,2024-08-20,Restaurants,Consumer Cyclical,2.748176e+09,5.55,0.384,0.02292,17.903227,3.047779e+09,8.482120e+08,1.804910e+08,1.804910e+08,2024-06-30,0.082058,0.065677


In [7]:
### Drop Utilities, Energy, and Financial Services  as suggested from the book ###
sectortoexclude = [
    'Utilities',
    'Energy',
    'Financial Services',
    'Real Estate'
]
for i in sectortoexclude:
    print('exclude sector > ',i)
    try:
         
        df = df.loc[df['sector'] != i]
    except:
        None
        
industrytoexclude =[
    'Engineering & Construction',
]
for i in industrytoexclude:
    print('exclude industry > ',i)
    try:
         
        df = df.loc[df['industry'] != i]
    except:
        None

df = df.reset_index(drop=True)

exclude sector >  Utilities
exclude sector >  Energy
exclude sector >  Financial Services
exclude sector >  Real Estate
exclude industry >  Engineering & Construction


In [8]:
#### Select market size by choosing ' market ' ###
market = 'SET'
df_market = df.loc[df['market'] == market]
df_market = df_market.reset_index(drop=True)

In [9]:
### Ranking regarding MFs####

df_market['Ranking_MF_ROC'] = df_market['MF_ROC'].rank()
df_market['Ranking_MF_EY'] = df_market['MF_EY'].rank()
df_market['Ranking_MF'] = df_market['Ranking_MF_ROC'] + df_market['Ranking_MF_EY']

df_market.loc[df_market['Ranking_MF'] == df_market['Ranking_MF'].max()]

,ticker,market,date_pulling,industry,sector,enterpriseValue,currentPrice,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
183,Q-CON.bk,SET,2024-08-20,Building Products & Equipment,Industrials,2.216408e+09,8.9,3.444,0.25512,4.68421,3.082648e+09,413558000.0,954431433.0,956501174.0,2024-06-30,0.358362,0.431555,293.0,296.0,589.0


In [10]:
numstocks = 50
df_sorted = df_market.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:numstocks]

,ticker,market,date_pulling,industry,sector,enterpriseValue,currentPrice,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,Q-CON.bk,SET,2024-08-20,Building Products & Equipment,Industrials,2.216408e+09,8.90,3.444,0.25512,4.684210,3.082648e+09,4.135580e+08,9.544314e+08,9.565012e+08,2024-06-30,0.358362,0.431555,293.0,296.0,589.0
1,UVAN.bk,SET,2024-08-20,Farm Products,Consumer Defensive,7.130260e+09,8.70,1.457,0.07948,7.250000,5.792787e+09,5.167410e+08,1.443106e+09,1.462381e+09,2024-06-30,0.277174,0.205095,285.0,279.0,564.0
2,PT.bk,SET,2024-08-20,Information Technology Services,Technology,2.681666e+09,9.05,0.078,0.07761,8.457944,2.410914e+09,1.457667e+09,3.886185e+08,3.988328e+08,2024-06-30,0.418394,0.148726,297.0,264.0,561.0
3,KBS.bk,SET,2024-08-20,Confectioners,Consumer Defensive,8.448462e+09,5.00,0.139,0.10625,2.336448,1.403379e+10,4.658636e+09,1.701101e+09,1.792721e+09,2024-06-30,0.191220,0.212195,263.0,284.0,547.0
4,AKR.bk,SET,2024-08-20,Electrical Equipment & Parts,Industrials,1.507079e+09,0.87,0.085,0.10124,5.800000,2.383655e+09,7.570691e+08,3.156780e+08,3.117502e+08,2024-06-30,0.191659,0.206857,264.0,282.0,546.0
5,CPW.bk,SET,2024-08-20,Electronics & Computer Distribution,Technology,1.775222e+09,2.54,0.168,0.01644,11.043478,1.949275e+09,9.017210e+08,1.707424e+08,2.656660e+08,2024-03-31,0.253606,0.149652,278.0,265.0,543.0
6,ICN.bk,SET,2024-08-20,Telecom Services,Communication Services,1.228253e+09,1.93,0.277,0.10458,7.423077,1.976144e+09,9.799810e+08,2.150695e+08,2.134139e+08,2024-06-30,0.214236,0.173754,270.0,272.0,542.0
7,BIZ.bk,SET,2024-08-20,Medical Devices,Healthcare,1.600064e+09,3.12,0.879,0.12233,10.758620,1.570356e+09,6.179430e+08,2.383282e+08,2.260287e+08,2024-06-30,0.237322,0.141262,276.0,252.0,528.0
8,UTP.bk,SET,2024-08-20,Paper & Paper Products,Basic Materials,5.289186e+09,11.10,2.874,0.20493,8.740158,5.175989e+09,4.462890e+08,9.613716e+08,8.489970e+08,2024-06-30,0.179503,0.160516,254.0,269.0,523.0
9,AS.bk,SET,2024-08-20,Electronic Gaming & Multimedia,Communication Services,1.769599e+09,3.68,0.707,0.18949,6.814815,1.406244e+09,4.061850e+08,3.326363e+08,2.331491e+08,2024-06-30,0.233135,0.131753,275.0,245.0,520.0


In [11]:
df_sorted_2 = df_sorted[:numstocks].copy()
df_sorted_2 = df_sorted_2.sort_values(by=['totalCashPerShare'],ascending=False)
df_sorted_2 = df_sorted_2.reset_index(drop=True)
df_sorted_2

,ticker,market,date_pulling,industry,sector,enterpriseValue,currentPrice,totalCashPerShare,profitMargins,trailingPE,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,METCO.bk,SET,2024-08-20,Electronic Components,Technology,2.093357e+09,184.50,98.090,0.03432,6.949153,9.200357e+09,2.138398e+09,8.879703e+08,7.319250e+08,2024-06-30,0.103643,0.349642,169.0,295.0,464.0
1,ALUCON.bk,SET,2024-08-20,Aluminum,Basic Materials,4.713384e+09,174.00,64.894,0.13389,9.354838,7.767908e+09,6.181490e+08,8.423445e+08,8.423445e+08,2024-06-30,0.117814,0.178713,192.0,273.0,465.0
2,KDH.bk,SET,2024-08-20,Medical Care Facilities,Healthcare,1.159295e+09,90.25,30.919,0.11634,13.757622,9.234060e+08,1.578040e+08,1.594004e+08,1.527765e+08,2024-06-30,0.199551,0.131784,268.0,246.0,514.0
3,SHANG.bk,SET,2024-08-20,Lodging,Consumer Cyclical,2.512884e+09,48.00,28.693,0.10131,26.815643,8.060359e+09,2.964750e+08,7.057970e+08,7.057970e+08,2024-06-30,0.090908,0.280871,150.0,293.0,443.0
4,NTV.bk,SET,2024-08-20,Medical Care Facilities,Healthcare,3.793487e+09,29.75,6.041,0.14949,12.713676,2.893072e+09,2.475280e+08,4.516894e+08,4.516894e+08,2024-06-30,0.170736,0.119070,242.0,233.0,475.0
5,PMTA.bk,SET,2024-08-20,Agricultural Inputs,Basic Materials,9.998570e+08,8.80,4.293,0.03474,6.027397,2.669535e+09,8.187940e+08,2.343163e+08,2.419880e+08,2024-06-30,0.130752,0.242023,207.0,290.0,497.0
6,MEGA.bk,SET,2024-08-20,Drug Manufacturers—Specialty & Generic,Healthcare,3.050456e+10,38.75,4.236,0.12746,16.847826,1.497095e+10,4.751162e+09,2.346631e+09,2.977635e+09,2024-06-30,0.291360,0.097613,287.0,209.0,496.0
7,TOA.bk,SET,2024-08-20,Specialty Chemicals,Basic Materials,2.944917e+10,17.60,3.659,0.11283,14.308943,2.133163e+10,5.077793e+09,3.306872e+09,3.098446e+09,2024-06-30,0.190629,0.105213,262.0,216.0,478.0
8,Q-CON.bk,SET,2024-08-20,Building Products & Equipment,Industrials,2.216408e+09,8.90,3.444,0.25512,4.684210,3.082648e+09,4.135580e+08,9.544314e+08,9.565012e+08,2024-06-30,0.358362,0.431555,293.0,296.0,589.0
9,SAT.bk,SET,2024-08-20,Auto Parts,Consumer Cyclical,3.674340e+09,11.80,3.287,0.09767,6.210527,9.987442e+09,1.537212e+09,9.310600e+08,8.486875e+08,2024-06-30,0.100434,0.230977,162.0,289.0,451.0
